In [1]:
# BoxModel - Geometry - Room

from dataclasses import dataclass, field
from honeybee.room import Room
from ladybug_geometry.geometry3d import Point3D
from honeybee.typing import clean_and_id_string
from honeybee.boundarycondition import boundary_conditions

# TODO - Add adiabatic toggle for roof / floor
# TODO - Add generation method for surrounding zones (good for IES export)

@dataclass
class BoxModelRoom:
    """Class for a typical BoxModel room with one external wall and remaining surfaces set to adiabatic"""
    name: str = field(init=True, default='BoxModel_Room')
    bay_width: float = field(init=True, default=3)
    count_bays: int = field(init = True, default=3)
    height: float = field(init=True, default=3)
    depth: float = field(init=True, default=10) 

    def __post_init__(self):
        # generate UUID
        self.identifier = clean_and_id_string(self.name)

        # Calculated variables
        # total box model width
        self.width = self.bay_width*self.count_bays
        # origin at center
        self.origin = Point3D(x=-self.width/2, y=-self.depth/2, z=0)
    
    @property
    def room(self) -> Room:
        """Returns a room from the BoxModel room geometry parameters"""
        room = Room.from_box(identifier = self.identifier,
                             width = self.width,
                             depth = self.depth,
                             height = self.height,
                             origin= self.origin)
        # set all faces to adiabatic
        for face in room.faces:
            face.boundary_condition = boundary_conditions.adiabatic
        # set north face (face 1) to outdoors, enables apertures to be added to this face 
        room.faces[1].boundary_condition = boundary_conditions.outdoors
        return room
        

In [2]:
# BoxModel - Geometry - Room
# TEST

room = BoxModelRoom().room
for face in room.faces:
    print(face.boundary_condition)

Adiabatic
Outdoors
Adiabatic
Adiabatic
Adiabatic
Adiabatic


In [3]:
# BoxModel - Geometry - Glazing

from dataclasses import dataclass, field
from honeybee.boundarycondition import Outdoors
from honeybee.facetype import Wall
from honeybee.shade import Shade
from honeybee.typing import clean_and_id_string

@dataclass
class BoxModelGlazing:
    """Class containing all the data required for BoxModel workflow"""
    glazing_ratio: float = field(init= True, default = 0.4)
    # targets may not be achieved, LBT will overide if needed to meet glazing ratio - TODO raise warning if not met?  
    target_window_height: float = field(init=True, default=2)
    target_sill_height: float = field(init=True, default=0.8)
    wall_thickness: float = field(init=True, default = 0.5)
    bay_width: float = field(init=True, default=3)

def assign_glazing_parameters(glazing_parameters: BoxModelGlazing, room: Room):
    """Returns a room with BoxModelGlazing parameters assigned, including adding wall thickness (reveal and internal wall finish)"""
    room = room.duplicate()
    for face in room.faces:
        if can_host_aperture(face):
            face.apertures_by_ratio_rectangle(ratio = glazing_parameters.glazing_ratio,
                                              aperture_height = glazing_parameters.target_window_height,
                                              sill_height = glazing_parameters.target_sill_height,
                                              horizontal_separation = glazing_parameters.bay_width,
                                              vertical_separation = 0)            

            if glazing_parameters.wall_thickness:
                for aperture in face.apertures:
                    assign_border_shades(aperture,
                                        depth = glazing_parameters.wall_thickness,
                                        indoor = True)
                    
                shade_identifier = clean_and_id_string('Internal_Face')
                internal_wall = Shade(identifier = shade_identifier,
                                    geometry= face.punched_geometry,
                                    is_detached = False)
                movement_vector = -face.normal*glazing_parameters.wall_thickness
                internal_wall.move(movement_vector)
                room.add_indoor_shade(internal_wall)
    return room

def can_host_aperture(face):
    """Test if a face is intended to host apertures (type:Wall & bc:Outdoors)"""
    return isinstance(face.boundary_condition, Outdoors) and \
        isinstance(face.type, Wall)

def assign_border_shades(aperture, depth, indoor):
    """Assign window border shades to an Aperture based on a set of inputs."""
    if isinstance(aperture.boundary_condition, Outdoors):
        aperture.extruded_border(depth, indoor)

In [13]:
# BoxModel - Geometry - Glazing
# TEST
glazing_parameters = BoxModelGlazing()
room_with_glazing = assign_glazing_parameters(glazing_parameters=glazing_parameters,
                                              room = room)
for shade in room_with_glazing.shades:
    print(shade)
room_with_glazing

Shade: Internal_Face_fa3eaccd


Room: BoxModel_Room_b062d923

In [17]:
# BoxModel - Model

from dataclasses import dataclass, field
from honeybee.model import Model
from honeybee.room import Room
from honeybee_vtk.model import Model as VTKModel
from honeybee.typing import clean_and_id_string

@dataclass
class BoxModelModel:
    @staticmethod
    def room_to_model(room: Room) -> Model:
        """Takes a single Room and returns a Model containing only that Room"""
        model_identifier = clean_and_id_string('BoxModel_Model')
        model = Model(identifier = model_identifier, rooms = [room])
        return model
    @staticmethod
    def model_to_vtkmodel(model: Model) -> VTKModel:
        """Takes a Model and returns a VTK version of it, for exporting to HTML and similar processes"""
        return VTKModel(model)

In [15]:
# BoxModel - Model
# TEST
model = BoxModelModel.room_to_model(room_with_glazing)
vtkmodel = BoxModelModel.model_to_vtkmodel(model)
vtkmodel.to_html(name = "Test", show=True)
model.to_gem()

ModelDataSet: Door has no data to be exported to folder.
ModelDataSet: AirBoundary has no data to be exported to folder.
ModelDataSet: Grid has no data to be exported to folder.


WindowsPath('BoxModel_Model_d75da6d2.gem')

In [18]:
# BoxModel - Construction

from ladybug.epw import EPW
from honeybee_energy.lib.constructionsets import construction_set_by_identifier
from honeybee.typing import clean_and_id_ep_string

def constr_set_from_base_climate_vintage_constr_type(epw: EPW, vintage: str, construction_type: str):
    """Returns a contruction set based on climate (EPW), ASHRAE vintage, and ASHRAE construction type"""
    climate_zone = epw.ashrae_climate_zone
    climate_zone = climate_zone[0]  # strip out any qualifiers like A, C, or C
    base_constr_set_str = '{}::{}{}::{}'.format(vintage, 'ClimateZone', climate_zone, construction_type)
    base_constr_set = construction_set_by_identifier(base_constr_set_str)
    constr_set = base_constr_set.duplicate()
    name = clean_and_id_ep_string('BoxModel_ConstructionSet')
    constr_set.identifier = name
    constr_set.unlock() # TODO what does this unlock do? 
    return constr_set

In [ ]:
# BoxModel - Construction - Opaque

from honeybee_energy.construction.opaque import OpaqueConstruction
from honeybee_energy.construction.shade import ShadeConstruction
from honeybee.typing import clean_and_id_ep_string

@dataclass
class BoxModelOpaqueInternal:
    base_constr: OpaqueConstruction
    int_sol_absorptance: float = field(init=True, default = 0.55) # Matches IES default
    int_therm_emissivity: float = field(init=True, default = 0.9) # Matches IES default
    int_vis_reflectance: float = field(init=True, default = 0.5)

    

@dataclass
class BoxModelOpaqueExternal(BoxModelOpaqueInternal):
    u_factor: float = field(init=True, default = 0.18) # TODO debate whether to match this to base set (ASHRAE) or set our own default
    ext_sol_absorptance: float = field(init=True, default = 0.7) # Matches IES default
    ext_therm_emissivity: float = field(init=True, default = 0.9) # Matches IES default
    ext_vis_reflectance: float = field(init=True, default = 0.3)

def adjust_vis_reflectance(construction: OpaqueConstruction,
                           inside_reflectance: Optional[float] = None,
                           outside_reflectance: Optional[float] = None):
    """Adjusts a constructions visible reflectance"""
    construction.unlock()
    if inside_reflectance is not None:
        construction.materials[-1].visible_reflectance = inside_reflectance
    if outside_reflectance is not None:
        construction.materials[0].visible_reflectance = outside_reflectance
    return construction

def adjust_sol_absorptance(construction: OpaqueConstruction,
                           inside_absorptance: Optional[float] = None,
                           outside_absorptance: Optional[float] = None):
    """Adjusts a constructions solar absorptance"""
    construction.unlock()
    if inside_absorptance is not None:
        construction.materials[-1].solar_absorptance = inside_absorptance
    if outside_absorptance is not None:
        construction.materials[0].solar_absorptance = outside_absorptance
    return construction

def adjust_thermal_absorptance(construction: OpaqueConstruction,
                               inside_absorptance: Optional[float] = None,
                               outside_absorptance: Optional[float] = None):
    """Adjusts a constructions thermal absorptance"""
    construction.unlock()
    if inside_absorptance is not None:
        construction.materials[-1].thermal_absorptance = inside_absorptance
    if outside_absorptance is not None:
        construction.materials[0].thermal_absorptance = outside_absorptance
    return construction

In [ ]:
# BoxModel - Construction - Opaque - Shade

@dataclass
class BoxModelShade:
    


In [ ]:
from honeybee_energy.construction.opaque import OpaqueConstruction
from ladybug.epw import EPW



def constr_set_from_base_climate_vintage(new_set_identifier: str, epw: EPW, vintage: str, construction_type: str):
    # Returns a contruction set based on climate (EPW), ASHRAE vintage, and ASHRAE construction type
    climate_zone = epw.ashrae_climate_zone
    climate_zone = climate_zone[0]  # strip out any qualifiers like A, C, or C
    constr_set = '{}::{}{}::{}'.format(vintage, 'ClimateZone', climate_zone, construction_type)
    base_constr_set = construction_set_by_identifier(constr_set)
    new_constr_set = base_constr_set.duplicate()
    new_constr_set.identifier = new_set_identifier
    new_constr_set.unlock()
    return new_constr_set




In [ ]:
Top Level Variable
name: str = field(init=True, default=None)
epw: Union[EPW, Path, str] # post init converts Path or str to EPW obj

Geometry - for room generation
bay_width: float = field(init=True, default=3)
count_bays: int = field(init = True, default=3)
height: float = field(init=True, default=3)
depth: float = field(init=True, default=10) 
orientation_angle: float = field(init= True, default = 180)

self.origin = Point3D(x=-self.width/2, y=-self.depth/2, z=0)
room = Room.from_box(identifier = self.name, width = self.width, depth=self.depth, height = self.height, origin=self.origin)
for face in room.faces:
    face.boundary_condition = boundary_conditions.adiabatic
# set north face (face 1) to outdoors, enables apertures to be added to this face 
room.faces[1].boundary_condition = boundary_conditions.outdoors

Window and reveal geometry
glazing_ratio: float = field(init= True, default = 0.4)
# targets may not be achieved, LBT will overide if needed to meet glazing ratio - TODO raise warning if not met?  
target_window_height: float = field(init=True, default=2)
target_sill_height: float = field(init=True, default=0.8)
wall_thickness: float = field(init=True, default = 0.5)

Shade (external) geometry

Energy - inputs (Room level)
construct_set: ConstructionSet = field(init=True, default=ConstructionSet(identifier='generic_constructions'))
program_type: ProgramType = field(init=True, default = ProgramType(identifier='generic_program_type'))

room.properties.energy.hvac = self.ideal_air_system
room.properties.energy.program_type = self.program_type

Radiance - inputs (Room level)
# This should always be a product of an energy input
modifier_set: ModifierSet = field(init=True, default=ModifierSet(identifier='generic_modifiers'))

room.properties.radiance.modifier_set = self.modifier_set
#Occupancy?

Sensor grid - Model level attribute
sensor_wall_offset: float = field(init=True, default = 0.5)
sensor_grid_size: float = field(init=True, default = 0.2)
sensor_grid_offset: float = field(init=True, default = 0.8)
sensor_grid_bay_count: int = field(init=True, default = 3)